My training of AB emus isn't working... going to explore

In [1]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
import numpy as np
from pearce.mocks.kittens import TrainingBox

In [3]:
cat = TrainingBox(boxno=0)

In [4]:
cat.load(1.0, HOD = 'hsabZheng07', hod_kwargs={'sec_haloprop_key': 'halo_local_density_5'})

hsabZheng07
HOD: hsabZheng07


/u/ki/swmclau2/.local/lib/python2.7/site-packages/halotools-0.7.dev5005-py2.7-linux-x86_64.egg/halotools/empirical_models/factories/hod_model_factory.py:741: UserWarning: 

The param_dict key sigma_logM appears in more than one component model.
This is permissible, but if you are seeing this message you should be sure you understand it.
In particular, double-check that this parameter does not have conflicting meanings across components.

If you do not wish to see this message every time you instantiate, 
simply attach a _suppress_repeated_param_warning attribute 
to any of your component models and set this variable to ``True``.

  warn(msg % key)
/u/ki/swmclau2/.local/lib/python2.7/site-packages/halotools-0.7.dev5005-py2.7-linux-x86_64.egg/halotools/empirical_models/factories/hod_model_factory.py:741: UserWarning: 

The param_dict key logMmin appears in more than one component model.
This is permissible, but if you are seeing this message you should be sure you understand it.
In parti

In [5]:
cat.halocat.halo_table['halo_local_density_10'].mean()

1.6254820559053744

In [6]:
default_params = cat.model.param_dict
default_params['mean_occupation_cens_assembias_param'] = 0.0
default_params['mean_occupation_sats_assembias_param'] = 0.0

In [7]:
cat.model.model_dictionary['centrals_occupation'].sec_haloprop_key

'halo_local_density_5'

In [ ]:
r_bins = np.logspace(-1.1, 1.6, 19)
rbc = (r_bins[1:]+r_bins[:-1])/2.0

In [ ]:
default_y = np.zeros((18,))
for i in xrange(10):
    print i,
    cat.populate(default_params)
    default_y+= cat.calc_xi(r_bins)
default_y/=10

/u/ki/swmclau2/.local/lib/python2.7/site-packages/halotools-0.7.dev5005-py2.7-linux-x86_64.egg/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:205: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_indices[digitized_param_list]
/u/ki/swmclau2/.local/lib/python2.7/site-packages/halotools-0.7.dev5005-py2.7-linux-x86_64.egg/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:522: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_in

0

In [ ]:
N = 3
cmap = sns.color_palette("GnBu_d", N)

In [ ]:
pname = 'mean_occupation_cens_assembias_param'
l, h = -1.0, 1.0
d = default_params.copy()
for v, c in zip(np.linspace(l,h,N), cmap):
    d[pname] = v
    y = np.zeros((18,))
    for i in xrange(10):
        print i,
        cat.populate(d)
        y+= cat.calc_xi(r_bins)
    y/=10
    print 
    plt.plot(rbc, y/default_y, label = v, c = c)
    
plt.xscale('log')
#plt.loglog()
plt.legend(loc = 'best')
plt.show();